In [19]:
import os
import shutil
import random
import numpy as np
import csv
import glob

In [26]:
# randomly choose 1 files from test set to make a baseline set
test_folder = "dataset/test"
baseline_folder = "baseline_set"

for subdir in os.listdir(test_folder):
    subdir_path = os.path.join(test_folder, subdir)
    if os.path.isdir(subdir_path):
        files = os.listdir(subdir_path)
        if files:
            file_to_copy = random.choice(files)
            source_path = os.path.join(subdir_path, file_to_copy)
            dest_path = os.path.join(baseline_folder, subdir + os.path.splitext(file_to_copy)[1])
            shutil.copy(source_path, dest_path)

In [1]:
# get list of classes name
path = 'dataset/test'
list_files = os.listdir(path)
list_files = np.array(list_files)

In [24]:
# make csv file that first column is classes name drop down (549 rows)
with open('baseline_list.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(list_files)

In [10]:
# get all img paths from baseline_set folder
def get_image_paths(folder_path):
    return sorted(glob.glob(os.path.join(folder_path, "*.jpg")))
folder_path = "baseline_set"
image_paths = get_image_paths(folder_path)

In [11]:
image_paths

['baseline_set\\Abalistes_stellatus.jpg',
 'baseline_set\\Abudefduf_saxatilis.jpg',
 'baseline_set\\Acanthemblemaria_spinosa.jpg',
 'baseline_set\\Acanthochromis_polyacanthus.jpg',
 'baseline_set\\Acanthurus_achilles.jpg',
 'baseline_set\\Acanthurus_chirurgus.jpg',
 'baseline_set\\Acanthurus_coeruleus.jpg',
 'baseline_set\\Acanthurus_dussumieri.jpg',
 'baseline_set\\Acanthurus_japonicus.jpg',
 'baseline_set\\Acanthurus_leucosternon.jpg',
 'baseline_set\\Acanthurus_lineatus.jpg',
 'baseline_set\\Acanthurus_maculiceps.jpg',
 'baseline_set\\Acanthurus_nigricans.jpg',
 'baseline_set\\Acanthurus_nigrofuscus.jpg',
 'baseline_set\\Acanthurus_nigroris.jpg',
 'baseline_set\\Acanthurus_olivaceus.jpg',
 'baseline_set\\Acanthurus_pyroferus.jpg',
 'baseline_set\\Acanthurus_sohal.jpg',
 'baseline_set\\Acanthurus_tennenti.jpg',
 'baseline_set\\Acanthurus_thompsoni.jpg',
 'baseline_set\\Acanthurus_triostegus.jpg',
 'baseline_set\\Acanthurus_tristis.jpg',
 'baseline_set\\Acanthurus_xanthopterus.jpg',
 

In [19]:
import requests
from google_img_source_search import ReverseImageSearcher

# upload to imgur bc the gg img search can be used with only image url
def upload_image_to_imgur(image_path, client_id):
    headers = {
        'Authorization': f'Client-ID {client_id}'
    }
    with open(image_path, 'rb') as image_file:
        response = requests.post(
            'https://api.imgur.com/3/image',
            headers=headers,
            files={'image': image_file}
        )
    response.raise_for_status()
    data = response.json()
    print('uploaded')
    return data['data']['link']

if __name__ == '__main__':
    x = 321
    image_path = image_paths[x]
    imgur_client_id = '7b5cacc1dfceaba' 

    # upload and get the url
    image_url = upload_image_to_imgur(image_path, imgur_client_id)

    # reverse img searching
    rev_img_searcher = ReverseImageSearcher()
    res = rev_img_searcher.search(image_url)
    print(image_paths[x])

    # print the result, title, site, img url
    for search_item in res:
        print(f'Title: {search_item.page_title}')
        print(f'Site: {search_item.page_url}')
        print(f'Img: {search_item.image_url}\n')


uploaded
baseline_set\Meuschenia_hippocrepis.jpg
Title: Meuschenia hippocrepis
Site: https://fishesofaustralia.net.au/home/species/807
Img: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTovxQFYayVM0vnmu1UFZ6LMGQTjWOTUvb-FOpowvYAdZL6A-Lo



In [29]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from PIL import Image, ImageFile
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [21]:
# import model
model_for_eva = tf.keras.models.load_model('model/model_group_mobile_04/ft10+5+4')

In [30]:
test_dir = 'baseline_set'

In [31]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [32]:
# get files list (class_name.jpg)
all_files = os.listdir(test_dir)

In [25]:
# get files path (x) and labels (y) as df
filepaths = [os.path.join(test_dir, f) for f in all_files]
labels = [f.split('.')[0] for f in all_files]

test_df = pd.DataFrame({
    'filename': filepaths,
    'class': labels
})

In [27]:
# test generator with x, y from df
test_gen = ImageDataGenerator(rescale=1./255)

test_generator = test_gen.flow_from_dataframe(
    test_df,
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    shuffle=False
)

Found 549 validated image filenames belonging to 549 classes.


In [28]:
# evaluate for the result
results = model_for_eva.evaluate(test_generator, verbose=1)

9/9 [==============================] - 14s 1s/step - loss: 0.6696 - accuracy: 0.8251
